In [69]:
import time
import requests

# Configuração da API e do ativo

SYMBOL = "ADA"  # Alterar para o símbolo desejado

# Função para obter os dados de profundidade
def get_order_book(symbol):
    BASE_URL = "https://api.coinw.com/v1/perpumPublic/depth"
    try:
        response = requests.get(BASE_URL, params={"base": symbol})
        if response.status_code == 200:
            data = response.json()
            if data.get("code") == 0:  # Código 0 indica sucesso na resposta
                return data.get("data")
            else:
                print(f"Erro na API: {data.get('msg', 'Mensagem desconhecida')}")
        else:
            print(f"Erro HTTP: {response.status_code}")
    except Exception as e:
        print(f"Erro ao acessar a API: {e}")
    return None

# # Loop principal
# def main():
#     try:
#         while True:
#             order_book = get_order_book(SYMBOL)
#             if order_book:
#                 print(f"Asks (Vendas): {order_book['asks']}")
#                 print(f"Bids (Compras): {order_book['bids']}")
#                 print(f"Profundidade de Asks: {len(order_book['asks'])} níveis")
#                 print(f"Profundidade de Bids: {len(order_book['bids'])} níveis")
#             else:
#                 print("Falha ao obter dados do order book.")
            
#             time.sleep(0.5)  # Intervalo de 0.5 segundos
#     except KeyboardInterrupt:
#         print("\nExecução interrompida pelo usuário.")

# if __name__ == "__main__":
#     main()


In [70]:
import time
import hmac
import hashlib
import base64
import requests
import json

# Configurações da API
BASE_URL = "https://api.coinw.com"
API_KEY = "230c2870-4c9a-41b9-af91-f2b8b3c85d9a"
SECRET_KEY = "91VNBM5QGSIBFU5W6GB2QZUIQIK17YBEIYFE"

def generate_signature(timestamp, method, endpoint, payload=None):
    """
    Gera a assinatura (sign) para autenticação na API da CoinW.

    Args:
        timestamp (str): Timestamp atual em milissegundos.
        method (str): Método HTTP ("GET" ou "POST").
        endpoint (str): Caminho do endpoint (ex.: "/v1/perpum/orders/open").
        payload (str ou dict, opcional): String de parâmetros ou JSON do payload.

    Returns:
        str: Assinatura gerada.
    """
    # Se o payload for um dicionário, converte para string formatada
    if isinstance(payload, dict):
        payload_str = "&".join(f"{key}={value}" for key, value in sorted(payload.items()))
    else:
        payload_str = payload or ""

    # Adiciona o '?' entre endpoint e parâmetros para GET
    if method.upper() == "GET" and payload_str:
        sign_param = f"{timestamp}{method.upper()}{endpoint}?{payload_str}"
    else:
        sign_param = f"{timestamp}{method.upper()}{endpoint}{payload_str}"

    # Gera a assinatura HMAC SHA256 e codifica em Base64
    signature = hmac.new(
        SECRET_KEY.encode('utf-8'),
        sign_param.encode('utf-8'),
        hashlib.sha256
    ).digest()
    return base64.b64encode(signature).decode()



In [71]:
# Método para obter as taxas de maker e taker
def get_fee_rate():
    """
    Consulta as taxas de maker e taker do usuário autenticado na CoinW.

    Returns:
        dict: Informações sobre as taxas ou erro, se aplicável.
    """
    endpoint = "/v1/perpum/account/fees"
    url = BASE_URL + endpoint
    timestamp = str(int(time.time() * 1000))
    
    # Gera a assinatura
    sign = generate_signature(timestamp, "GET", endpoint, "")
    
    # Cabeçalhos da requisição
    headers = {
        "Content-Type": "application/json",
        "timestamp": timestamp,
        "api_key": API_KEY,
        "sign": sign
    }
    
    try:
        # Requisição GET
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get("code") == 0:  # Código 0 indica sucesso
                return {
                    "makerFee": data["data"]["makerFee"],
                    "takerFee": data["data"]["takerFee"],
                    "userId": data["data"]["userId"]
                }
            else:
                return {"status": "error", "message": data.get("msg", "Erro desconhecido")}
        else:
            return {"status": "http_error", "code": response.status_code}
    except Exception as e:
        return {"status": "error", "message": str(e)}

# Exemplo de uso
if __name__ == "__main__":
    fee_rate = get_fee_rate()
    if "makerFee" in fee_rate:
        print("Taxas obtidas com sucesso:")
        print(f"Maker Fee: {fee_rate['makerFee']}")
        print(f"Taker Fee: {fee_rate['takerFee']}")
        print(f"User ID: {fee_rate['userId']}")
    else:
        print(f"Erro ao obter as taxas: {fee_rate}")

Taxas obtidas com sucesso:
Maker Fee: 0.0001
Taker Fee: 0.0006
User ID: 1164959


In [72]:
def get_positions(symbol, open_ids=None):
    """
    Consulta informações de posições abertas para o ativo especificado.

    Args:
        symbol (str): Símbolo do ativo (ex: BTC).
        open_ids (list, opcional): Lista de IDs de posições abertas.

    Returns:
        dict: Informações sobre as posições abertas ou mensagem de erro.
    """
    endpoint = "/v1/perpum/positions"
    url = BASE_URL + endpoint
    timestamp = str(int(time.time() * 1000))

    # Parâmetros obrigatórios e opcionais
    params = {"instrument": symbol}
    if open_ids:
        params["openIds"] = ",".join(map(str, open_ids))  # Lista separada por vírgulas

    # Ordena os parâmetros para assinatura
    payload_str = "&".join(f"{key}={value}" for key, value in sorted(params.items()))
    sign = generate_signature(timestamp, "GET", endpoint, payload_str)

    # Cabeçalhos da requisição
    headers = {
        "Content-Type": "application/json",
        "timestamp": timestamp,
        "api_key": API_KEY,
        "sign": sign,
    }

    try:
        # Envia a requisição GET
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get("code") == 0:  # Código 0 indica sucesso
                return data.get("data", [])
            else:
                return {"status": "error", "message": data.get("msg", "Erro desconhecido")}
        else:
            return {"status": "http_error", "code": response.status_code}
    except Exception as e:
        return {"status": "error", "message": str(e)}


In [73]:
positions = get_positions("ADA")
print("Posições Abertas:", positions)

Posições Abertas: []


In [74]:
def cancel_order(order_id):
    """
    Cancela uma ordem existente na CoinW.

    Args:
        order_id (str): ID da ordem a ser cancelada.

    Returns:
        dict: Resposta da API.
    """
    endpoint = "/v1/perpum/order"
    url = BASE_URL + endpoint
    timestamp = str(int(time.time() * 1000))

    # Corpo da requisição
    payload = {"id": order_id}

    # Gera a assinatura
    sign = generate_signature(timestamp, "DELETE", endpoint, json.dumps(payload))

    # Cabeçalhos da requisição
    headers = {
        "Content-Type": "application/json",
        "timestamp": timestamp,
        "api_key": API_KEY,
        "sign": sign
    }

    # Envia a requisição DELETE
    response = requests.delete(url, json=payload, headers=headers)

    # Processa a resposta
    if response.status_code == 200:
        return response.json()
    else:
        return {"status": "http_error", "code": response.status_code, "message": response.text}

def has_open_order(symbol):
    """
    Verifica se há ordens abertas para um determinado ativo.

    Args:
        symbol (str): Símbolo do ativo (ex: BTC).

    Returns:
        list: Lista de ordens abertas ou erro.
    """
    endpoint = "/v1/perpum/orders/open"
    url = BASE_URL + endpoint
    timestamp = str(int(time.time() * 1000))

    # Parâmetros da requisição
    params = {
        "instrument": symbol,
        "positionType": "PostOnly"  # Tipo de ordem que estamos verificando
    }
    payload_str = "&".join(f"{key}={value}" for key, value in sorted(params.items()))

    # Gera a assinatura
    sign = generate_signature(timestamp, "GET", endpoint, payload_str)

    # Cabeçalhos da requisição
    headers = {
        "Content-Type": "application/json",
        "timestamp": timestamp,
        "api_key": API_KEY,
        "sign": sign
    }

    # Envia a requisição GET
    response = requests.get(url, params=params, headers=headers)

    # Processa a resposta
    if response.status_code == 200:
        data = response.json()
        if data.get("code") == 0:
            return data["data"]["rows"]  # Retorna a lista de ordens abertas
        else:
            return {"status": "error", "message": data.get("msg", "Erro desconhecido")}
    else:
        return {"status": "http_error", "code": response.status_code, "message": response.text}

     


In [75]:
# Função para pegar o order book
def get_order_book(symbol):
    """
    Obtém o preço mais alto de compra (bid) e o mais baixo de venda (ask) do order book.
    """
    try:
        response = requests.get(f"{BASE_URL}/v1/perpumPublic/depth", params={"base": symbol})
        if response.status_code == 200:
            data = response.json()
            if data.get("code") == 0:
                bids = data['data']['bids']
                asks = data['data']['asks']
                best_bid = float(bids[0]['p']) if bids else None
                best_ask = float(asks[0]['p']) if asks else None
                return best_bid, best_ask
        else:
            print(f"Erro ao acessar o order book: {response.status_code}")
    except Exception as e:
        print(f"Erro ao acessar a API: {e}")
    return None, None

In [76]:
# Função para abrir uma ordem a limite
def place_order(
    instrument, direction, leverage, quantity_unit, quantity,
    position_model, position_type="execute", open_price=None,
    stop_loss_price=None, take_profit_price=None
):
    """
    Envia uma ordem autenticada para a API da CoinW.
    """
    endpoint = "/v1/perpum/order"
    url = BASE_URL + endpoint
    timestamp = str(int(time.time() * 1000))

    payload = {
        "instrument": instrument,
        "direction": direction,
        "leverage": leverage,
        "quantityUnit": quantity_unit,
        "quantity": quantity,
        "positionModel": position_model,
        "positionType": position_type,
    }

    if open_price:
        payload["openPrice"] = open_price
    if stop_loss_price:
        payload["stopLossPrice"] = stop_loss_price
    if take_profit_price:
        payload["stopProfitPrice"] = take_profit_price

    payload_str = json.dumps(payload, separators=(',', ':'))
    sign = generate_signature(timestamp, "POST", endpoint, payload_str)

    headers = {
        "Content-Type": "application/json",
        "timestamp": timestamp,
        "api_key": API_KEY,
        "sign": sign
    }

    try:
        response = requests.post(url, headers=headers, data=payload_str)
        return response.json()
    except Exception as e:
        return {"status": "error", "message": str(e)}
        
def place_near_market_maker_order_with_retry(symbol, direction, leverage, quantity):
    """
    Abre uma posição maker o mais próximo possível do preço atual, com um loop
    que cancela e reenvia a ordem até que ela seja acionada.

    Args:
        symbol (str): Símbolo do ativo (ex: BTC).
        direction (str): Direção da ordem ("long" ou "short").
        leverage (int): Alavancagem.
        quantity (str): Quantidade da ordem.

    Returns:
        dict: Resposta da API final quando a ordem for preenchida.
    """
    while True:
        # Obtenha o bid e ask mais recentes
        bid, ask = get_order_book(symbol)

        if not bid or not ask:
            print("Erro ao obter o livro de ofertas. Tentando novamente...")
            time.sleep(1)
            continue

        # Defina o preço limite próximo ao mercado
        if direction == "long":
            open_price = ask - 0.0001  # Compra: ligeiramente abaixo do ask
        elif direction == "short":
            open_price = bid + 0.0001  # Venda: ligeiramente acima do bid
        else:
            return {"status": "error", "message": "Direção inválida. Use 'long' ou 'short'."}

        # Envia a ordem PostOnly
        response = place_order(
            instrument=symbol,
            direction=direction,
            leverage=leverage,
            quantity_unit=0,  # Unidade: 0 para USDT | 1 para contratos
            quantity=quantity,
            position_model=0,  # Isolado
            position_type="PostOnly",  # Configura como maker
            open_price=f"{open_price:.4f}"  # Preço limite próximo ao mercado
        )
        print("Ordem enviada:", response)

        # Verifique se a ordem foi aceita
        if response.get("code") != 0:
            print("Erro ao enviar ordem:", response.get("msg", "Erro desconhecido"))
            time.sleep(1)
            continue

        order_id = response["data"]["value"]

        # Aguarde 1 segundo para verificar se a ordem foi preenchida
        time.sleep(1)

        # Verifique se a ordem ainda está aberta
        positions = get_positions("ADA")
        if len(positions) > 0:
            print("Posicao acionada com sucesso!")
            return response  # Ordem foi preenchida

        # Cancela a ordem se ela ainda estiver aberta
        cancel_response = cancel_order(order_id)
        print("Ordem cancelada:", cancel_response)

        # Aguarde um momento antes de tentar novamente
        time.sleep(1)


In [77]:
response = place_near_market_maker_order_with_retry(
    symbol="ADA",
    direction="long",
    leverage=1,
    quantity="15"
)
print("Resposta Final (Venda):", response)

Ordem enviada: {'code': 0, 'data': {'value': '33308683783540534'}, 'msg': ''}
Posicao acionada com sucesso!
Resposta Final (Venda): {'code': 0, 'data': {'value': '33308683783540534'}, 'msg': ''}


In [78]:
def cancel_all_open_orders(symbol):
    """
    Cancela todas as ordens abertas para um símbolo.

    Args:
        symbol (str): Símbolo do ativo (ex: BTC).

    Returns:
        dict: Resposta da API para a operação de cancelamento.
    """
    endpoint = "/v1/perpum/orders/open"
    orders = has_open_order(symbol)
    if not orders:
        print(f"Não há ordens abertas para {symbol}.")
        return {"status": "success", "message": "Nenhuma ordem aberta para cancelar."}

    for order in orders:
        order_id = order["id"]
        cancel_response = cancel_order(order_id)
        print(f"Ordem cancelada (ID: {order_id}):", cancel_response)

    return {"status": "success", "message": "Todas as ordens abertas foram canceladas."}


In [79]:
cancel_all_open_orders("ADA")

Não há ordens abertas para ADA.


{'status': 'success', 'message': 'Nenhuma ordem aberta para cancelar.'}

In [82]:
def close_position_with_tpsl(symbol, position_id, direction, quantity):
    """
    Fecha uma posição aberta usando a API de Take Profit e Stop Loss (TPSL) como maker.

    Args:
        symbol (str): Símbolo do ativo (ex: ADA).
        position_id (int): ID da posição a ser encerrada.
        direction (str): Direção da posição aberta ("long" ou "short").
        quantity (str): Quantidade da posição a ser encerrada.

    Returns:
        dict: Resposta da API para a configuração do TP/SL.
    """
    # Obtenha o bid e ask mais recentes
    bid, ask = get_order_book(symbol)

    if not bid or not ask:
        return {"status": "error", "message": "Erro ao obter o livro de ofertas."}

    # Configure os preços de TP e SL como maker
    if direction == "long":
        stop_profit_order_price = ask + 0.005  # TP ligeiramente acima do ask
        stop_loss_order_price = bid - 0.005    # SL ligeiramente abaixo do bid
    elif direction == "short":
        stop_profit_order_price = bid - 0.005  # TP ligeiramente abaixo do bid
        stop_loss_order_price = ask + 0.005    # SL ligeiramente acima do ask
    else:
        return {"status": "error", "message": "Direção inválida. Use 'long' ou 'short'."}

    # Corpo da requisição para configurar o TPSL
    payload = {
        "id": position_id,
        "stopProfitPrice": f"{stop_profit_order_price:.4f}",
        "stopLossPrice": f"{stop_loss_order_price:.4f}"
    }

    # Endpoint e cabeçalhos
    endpoint = "/v1/perpum/TPSL"
    url = BASE_URL + endpoint
    timestamp = str(int(time.time() * 1000))
    sign = generate_signature(timestamp, "POST", endpoint, json.dumps(payload))

    headers = {
        "Content-Type": "application/json",
        "timestamp": timestamp,
        "api_key": API_KEY,
        "sign": sign
    }

    # Envia a requisição POST
    response = requests.post(url, json=payload, headers=headers)

    # Processa a resposta
    if response.status_code == 200:
        return response.json()
    else:
        return {"status": "http_error", "code": response.status_code, "message": response.text}


In [87]:
positions = get_positions("ADA")
for pos in positions:
    if pos["status"] == "open" and pos["direction"] == "short":  # Exemplo para posição short
        print("Detalhes da posição aberta:", pos)
        position_id = pos["id"]
        quantity = pos["quantity"]
positions

[{'autoDeleveragingScore': 1,
  'base': 'ada',
  'baseSize': 10.0,
  'closedPiece': 0,
  'createdDate': 1736187320000,
  'currentPiece': 1,
  'direction': 'long',
  'fee': '0.001',
  'fundingFee': '0',
  'fundingSettle': 0,
  'id': 2435521222630648154,
  'indexPrice': 1.094,
  'instrument': 'ADA',
  'leverage': 1,
  'margin': 10.94,
  'openPrice': 1.094,
  'orderPrice': 1.094,
  'originalType': 'plan',
  'posType': 'execute',
  'positionMargin': 10.94,
  'positionModel': 0,
  'profitReal': 0,
  'profitUnreal': 0.002,
  'quantity': 10.94,
  'quantityUnit': 0,
  'source': 'api',
  'status': 'open',
  'totalPiece': 1,
  'updatedDate': 1736187320000,
  'userId': 1164959}]

In [86]:
response = close_position_with_tpsl(
    symbol="ADA",
    position_id=position_id,
    direction="long",  # Direção da posição aberta
    quantity=quantity
)
print("Resposta TPSL (Fechamento de Posição Short):", response)

Resposta TPSL (Fechamento de Posição Short): {'code': 9012, 'msg': 'Position not found.'}


In [36]:
response = close_position_with_tpsl(
    symbol="ADA",
    position_id=position_id,
    direction="long",  # Direção da posição aberta
    quantity=quantity
)
print("Resposta TPSL (Fechamento de Posição Long):", response)

Resposta TPSL (Fechamento de Posição Long): {'code': 0, 'msg': 'Success'}
